In [59]:
# activate package, basic imports + load data
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_fixtures.jl")
include("/Users/federicoclaudi/Documents/Github/LocomotionControl/analysis/behavior/analysis_visuals.jl")

  Activating project at `~/Documents/Github/LocomotionControl/analysis/behavior`


@Info (Main):  Loaded 1021 trials 
  ╰──────────────────────────────────────────────── 
                      Fri, 20 May 2022 12:07:31 


kdeplot!

In [60]:
import jcontrol.bicycle: Bicycle
import jcontrol.control: Bounds, ControlOptions
import InfiniteOpt: termination_status
import jcontrol.Run: run_mtm
import jcontrol: toDict

# Params effect
Vary a bunch of params and run global MTM to assess effect of parameters change


## Prepare parameters set.

In [61]:
savepath = "/Users/federicoclaudi/Dropbox (UCL)/Rotation_vte/Locomotion/analysis/behavior/mtm_parameters"

p = -1:.1:1
@info "Number of values per parameter: $(length(p))"

@Info (Main):  Number of values per parameter: 21 
  ╰──────────────────────────────────────────────── 
                      Fri, 20 May 2022 12:07:31 


In [62]:
bike_default = Dict(
    :l_f=>3,
    :l_r=>2,
    :width=>1.8,
    :m_f=>10, 
    :m_r=>12, 
    :c=>4e3
)

bike_Δ = Dict(
    :l_f=>2,
    :l_r=>2,
    :width=>.2,
    :m_f=>3, 
    :m_r=>3, 
    :c=>2e3
)

@info "Tot N simulations for bike: $(length(p) * length(bike_Δ))"

@Info (Main):  Tot N simulations for bike: 126 
  ╰──────────────────────────────────────────────── 
                      Fri, 20 May 2022 12:07:31 


## Run simulation

In [63]:
count = 1
for (parameter, pΔ) in pairs(bike_Δ)
    @info "Running simulations for: $(parameter)"
    for pval in p
        kwargs = (parameter => bike_default[parameter] + pΔ * pval, )
        bike = Bicycle(; kwargs...)

        _, _, control_model, solution = run_mtm(
            :dynamics,  # model type
            2.0;  # supports density
            showtrials=nothing,
            showplots=false,
            quiet=true,
            bike=bike
        )
        
        if "LOCALLY_SOLVED" == string(termination_status(control_model))
            destination = joinpath(savepath, "bike_param_$count.csv")
            data = DataFrame(toDict(solution))
            CSV.write(destination, data)
        else
            @warn "Failed" termination_status(control_model)
        end
        count += 1
    end
end

## Make parameters values metadata

In [ ]:
import OrderedCollections: OrderedDict
params_metadata = Dict{Int64, Pair}()

count = 1
for (parameter, pΔ) in pairs(bike_Δ)
    for pval in p
        params_metadata[count] =  parameter => bike_default[parameter] + pΔ * pval

        count += 1
    end
end

"""
For each parameter get the ID of all the simulations that 
varied that parameter values
"""
function get_param_runs_ids(param::Symbol)
    res = OrderedDict{Int64, Number}()

    for (id, (sim_param, value)) in pairs(params_metadata)
       if sim_param == param
        res[id] = value
       end
    end

   return sort!(res)
end


## Analyze

In [ ]:
solutions, _names = load_mtm_solutions(; folder=savepath, name="")
@info "Loaded $(length(solutions)) solutions"
solutions_ids = map(n -> parse(Int64, split(split(n, "_")[3], ".")[1]), _names);

solutions = Dict(_id=>sol for (_id, sol) in zip(solutions_ids, solutions))

In [ ]:
get_param_runs_ids(:width)

In [ ]:
function make_palette(x)
    return range(HSL(217, .64, .55), stop=HSL(320, .73, .78), length=length(x))
end

In [ ]:
plots = []
colors = make_palette(p)

for para in keys(bike_Δ)
    runs_ids = get_param_runs_ids(para)

    param_solutions = [(get(solutions, _id, nothing), val) for (_id, val) in runs_ids]

    plt = draw(:arena)
    for (n, (sol, val)) in enumerate(param_solutions)
        isnothing(sol) && continue
        plot!(sol.x, sol.y, 
            lw=2, color=colors[n],
            label="$(para): $(round(val; digits=2)) - ($(round(sol.t[end]; digits=2)), s)")
    end
    plot!(title=para)
    push!(plots, plt)

end

plot(plots...; grid=false)